<a href="https://colab.research.google.com/github/Pratik325/CNN/blob/master/project_half.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTING STATEMENTS**

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


**DATAFRAME**

In [0]:
df_global = pd.read_csv('/content/drive/My Drive/project.csv')

**PRINITNG THE SHAPE**

In [3]:
df_global.shape

(18222, 4097)

**SPLITTING THE DATA**

In [4]:
X = df_global.drop('labels', axis = 1)
y = df_global['labels']
print(X.shape)
print(y.shape)

(18222, 4096)
(18222,)


**IMPORTING AND NORMALIZATIONS**

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y ,test_size = 0.4)

# Normalizing the RGB codes by dividing it to the max RGB value.
x_train = x_train/255.
x_test = x_test/255.


In [6]:
print(x_train.shape)

(10933, 4096)


In [7]:
print(x_test.shape)

(7289, 4096)


**CONVERTING TO DESIRED SHAPE REQUIRED BY THE KERAS API**

In [0]:

list1 = []
arr1 = np.array([])
x_train = x_train.to_numpy()
for i in range(len(x_train)):
    list1.append(x_train[i].reshape(64,64))


In [9]:
x_train = np.array(list1)
x_train.shape

(10933, 64, 64)

In [10]:
list2 = []
arr2 = np.array([])
x_test = x_test.to_numpy()
for i in range(len(x_test)):
    list2.append(x_test[i].reshape(64,64))

x_test = np.array(list2)
x_test.shape   #printing the shape

(7289, 64, 64)

In [11]:
print(y_train.shape)
print(y_test.shape)
# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 64, 64, 1)
x_test = x_test.reshape(x_test.shape[0], 64, 64, 1)

(10933,)
(7289,)


In [0]:
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

**CREATING THE MODEL**

In [13]:
# Importing the required Keras modules containing model and layers
import tensorflow as tf
from tensorflow import keras
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, Input
from keras.optimizers import SGD


Using TensorFlow backend.


In [14]:
def create_model():
  def add_conv_block(model, num_filters):
    model.add(Conv2D(num_filters, 3, activation='relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(num_filters, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.5))
    return model
  
  model = tf.keras.models.Sequential()
  model.add(Input(shape = (64, 64, 1)))

  model = add_conv_block(model, 32)
  model = add_conv_block(model, 64)
  model = add_conv_block(model, 128)

  model.add(Flatten())
  model.add(Dense(2, activation='softmax'))

  model.compile(
      loss='sparse_categorical_crossentropy',
      optimizer=keras.optimizers.Adam( learning_rate=0.001,beta_1 = 0.9,beta_2 = 0.999,epsilon = 1e-8), metrics = ['accuracy']
  )
  return model
          
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 31, 64)        2

In [15]:
h = model.fit(
    x_train, y_train,
    validation_data = (x_test, y_test),
    epochs = 10, batch_size = 128,
    callbacks = [   
        tf.keras.callbacks.ModelCheckpoint('model_new/model_{val_accuracy:.3f}.h5',
                                           save_best_only = True, save_weights_only = False,
                                           monitor = 'val_accuracy'
                                           )

    ]
)

Epoch 1/10
86/86 [==============================] - 9s 107ms/step - loss: 1.0955 - accuracy: 0.6067 - val_loss: 1.1301 - val_accuracy: 0.4991
Epoch 2/10
86/86 [==============================] - 8s 96ms/step - loss: 0.7341 - accuracy: 0.6613 - val_loss: 1.4511 - val_accuracy: 0.4991
Epoch 3/10
86/86 [==============================] - 8s 95ms/step - loss: 0.5925 - accuracy: 0.6972 - val_loss: 1.7806 - val_accuracy: 0.4991
Epoch 4/10
86/86 [==============================] - 8s 95ms/step - loss: 0.5484 - accuracy: 0.7247 - val_loss: 2.8445 - val_accuracy: 0.4991
Epoch 5/10
86/86 [==============================] - 8s 94ms/step - loss: 0.5345 - accuracy: 0.7334 - val_loss: 3.3139 - val_accuracy: 0.4991
Epoch 6/10
86/86 [==============================] - 8s 94ms/step - loss: 0.5162 - accuracy: 0.7442 - val_loss: 2.5832 - val_accuracy: 0.4991
Epoch 7/10
86/86 [==============================] - 8s 94ms/step - loss: 0.4954 - accuracy: 0.7578 - val_loss: 3.4062 - val_accuracy: 0.4991
Epoch 8/10
8

In [0]:
pred = model.predict(x_test)

**Model Evaluation on the Test Set**

In [0]:
test_eval = model.evaluate(x_test, y_test, verbose=0)

In [18]:

print('Test accuracy:', test_eval[1])


Test accuracy: 0.49924543499946594


**AS THE TEST LOSS IS MORE LETS TRY HYPERPARAMETER TUNING**

**USING KERAS TUNNER**

In [19]:
!pip install keras-tuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,3] ),
        activation='relu',
        input_shape=(64,64,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,3]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(2, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [21]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5)

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [22]:
tuner_search.search(x_train,y_train,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 172800)            0         
_________________________________________________________________
dense (Dense)                (None, 48)                8294448   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 98        
Total params: 8,344,098
Trainable params: 8,344,098
Non-trainable params: 0
_________________________________________________________________


In [25]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.0001)
h2 = model.fit(x_train, y_train, epochs=20, validation_split=0.1, callbacks =[reduce_lr], batch_size=128)

Epoch 1/20
77/77 [==============================] - 10s 129ms/step - loss: 0.5705 - accuracy: 0.6711 - val_loss: 0.5780 - val_accuracy: 0.6773 - lr: 0.0010
Epoch 2/20
77/77 [==============================] - 9s 117ms/step - loss: 0.5465 - accuracy: 0.6919 - val_loss: 0.5837 - val_accuracy: 0.6792 - lr: 0.0010
Epoch 3/20
77/77 [==============================] - 9s 117ms/step - loss: 0.5210 - accuracy: 0.7116 - val_loss: 0.5805 - val_accuracy: 0.6846 - lr: 0.0010
Epoch 4/20
77/77 [==============================] - 9s 117ms/step - loss: 0.4792 - accuracy: 0.7350 - val_loss: 0.5931 - val_accuracy: 0.6737 - lr: 0.0010
Epoch 5/20
77/77 [==============================] - 9s 116ms/step - loss: 0.4280 - accuracy: 0.7676 - val_loss: 0.6148 - val_accuracy: 0.6801 - lr: 2.0000e-04
Epoch 6/20
77/77 [==============================] - 9s 116ms/step - loss: 0.4089 - accuracy: 0.7783 - val_loss: 0.6293 - val_accuracy: 0.6764 - lr: 2.0000e-04
Epoch 7/20
77/77 [==============================] - 9s 116ms/

In [26]:
pred = model.predict(x_test)
test_eval = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', test_eval[1])

Test accuracy: 0.6836328506469727
